### Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from time import sleep

### Problem Definition


In [2]:
# Problem Parameters and Cost Function Declaration

NFE = 0         # Number of Function Evaluation

fname = r"pid_gain_err.txt"
f_s = 100
t_s = 1 / f_s
setpoint = 0
max_speed = 3400
throttle = 2700

def rms(data):
    return np.sqrt(np.mean(data ** 2))
    

# def pidGainEval0(gains):
    
#     global NFE
    
#     # Run the Simulation
#     os.system(f"./pidGainEval {throttle} {gains[0]} {gains[1]} {gains[2]} {gains[3]} {max_speed}")
    
#     data = np.loadtxt(fname=fname)  # Read Data
    
#     if (np.sum(data == 10000) > 0):
#         cost = rms(data) * 1000   # Return a Big Cost
        
#     else:
#         rms_data = rms(data - setpoint)                # Calculating RMS of the Signal
#         peak = np.max(data) - setpoint                 # Calculating Peak
        
#         # Calculating the Rising Time
#         minTime = 0.1 * setpoint
#         maxTime = 0.9 * setpoint
#         try:
#             startInd = np.argwhere(data >= minTime)[0]
#             endInd = np.argwhere(data <= maxTime)[0]
            
#             settleTime = t_s * (endInd - startInd)   # Convert index into time
#             print('try')
#             print(f" start Idx = {startInd}, End Idx = {endInd}, t_s = {t_s}")
            
#         except:
#             settleTime = data.size * t_s
#             print('except')
        
#         cost = rms_data * peak * settleTime  # Calculate the Cost
        
#         print(rms_data, peak, settleTime)
              
#     NFE += 1                # Add to the function evaluation Count
    
#     os.system(f"mv pid_gain_err.txt flightData/{NFE}.txt")  # Moving the File into the Folder 
    
#     sleep(2)                    # Wait for 2 Seconds
    
#     return cost

def pidGainEval(gains):
    
    global NFE
    
    # Run the Simulation
    os.system(f"./pidGainEval {throttle} {gains[0]} {gains[1]} {gains[2]} {gains[3]} {max_speed}")
    
    data = np.loadtxt(fname=fname)  # Read Data
    
    if (np.sum(data == 10000) > 0):
        cost = rms(data) * 1000   # Return a Big Cost
        
    else:
        cost = rms(data - setpoint)                # Calculating RMS of the Signal
                      
    NFE += 1                # Add to the function evaluation Count
    
    os.system(f"mv pid_gain_err.txt flightData/{NFE}.txt")  # Moving the File into the Folder 
    sleep(2)                    # Wait for 2 Seconds
    
    return cost

In [3]:
gains = [5, .2, 3, 2]
print(pidGainEval(gains))

10.90628904004474


In [4]:
cost_func = pidGainEval
n_var = 4                                        # Number of decision Vars
var_size = np.array([1, n_var])                  # Decision Variable Mat Size
var_range = np.array([30, 1.5, 10, 5])           # Desicion Var Lower Bound

### BBO Parameters


In [5]:
max_it = 20                     # Max Number of Iterations
n_pop = 5                       # Number of Habitats

keep_rate = 0.2                         # Keep Rate (Elitism Rate)
n_keep = round(keep_rate * n_pop)       # Number of Kept Habitats (No of Elits)
n_new = n_pop - n_keep                  # Number of New Habitats

### Migeration Rates

In [6]:
mu = np.linspace(1, 0, n_pop)           # Emmigration Rates
lmbda = np.linspace(0, 1, n_pop)        # Immigration Rates

alpha = 0.9                             # Linear Enterpolation Coefficient
p_mutation = 0.05                       # Mutation Rate
sigma = 0.05 * (var_range * 2)          # Standard Devision

### Initialization

In [7]:
pop_position = np.zeros((n_pop, n_var))
pop_cost = np.zeros((n_pop))


for iter in range(0, n_pop):
    
    for j in  range(n_var):
        pop_position[iter, j] = np.random.uniform(
            -var_range[j], var_range[j], 1)
        
    pop_cost[iter] = cost_func(pop_position[iter, :])

# Function to Sort Population due to Costs
def pop_sort(pos, cost):
    sort_order = np.argsort(cost, axis=0)

    cost = cost[sort_order]
    pos = pos[sort_order]

    return pos, cost

# Sort Population
pop_position, pop_cost = pop_sort(pop_position, pop_cost)

# Store the Best Solution Ever Found
best_sol = pop_position[0, :]
best_cost = np.zeros((max_it))              # Array to Hold Best Costs

# Roulette Wheel Selection (Selection due to Probabilities)1
def roulette_wheel_selection(prob):
    r = np.random.rand() * np.sum(prob)
    cumsum = 0
    for i in range(len(prob)):
        cumsum += prob[i]
        if (cumsum > r):
            return i

In [8]:
max_speed = 3200

### BBO Main Loop

In [9]:
for iter in range(max_it):
    
    # Create a Copy of the Population
    new_pop_cost = pop_cost
    new_pop_pos = pop_position

    for i in range(n_pop):
        for k in range(n_var):
            # Migration
            if np.random.rand() <= lmbda[i]:
                # Modify Emmigration Probabilities
                EP = mu
                EP[i] = 0
                EP /= np.sum(EP)                # Normalize it!
                
                # Select Source Habitat
                j = roulette_wheel_selection(EP)
                
                # Migration
                new_pop_pos[i, k] = pop_position[i, k] 
                + alpha * (pop_position[j, k] - pop_position[i, k])
            
            # Mutation
            if np.random.rand() <= p_mutation:
                new_pop_pos[i, k] += sigma[k] * np.random.randn(1)
            
        # Evaluation
        new_pop_cost[i] = cost_func(new_pop_pos[i, :])
    
    # Sort New Population
    new_pop_pos, new_pop_cost = pop_sort(new_pop_pos, new_pop_cost)
    
    # Select Next Iteration Population
    pop_position = np.concatenate((pop_position[0:n_keep, :], new_pop_pos[0:n_new, :]), axis=0)
    pop_cost = np.concatenate((pop_cost[0:n_keep], new_pop_cost[0:n_new]), axis = 0)
    
    # Sort Population
    pop_position, pop_cost = pop_sort(pop_position, pop_cost)
    
    # Update Best Solution Ever Found
    best_sol = pop_position[0, :]
    
    # Store Best Cost Ever Found
    best_cost[iter] = pop_cost[0]
    
    # Log Iteration Info
    print(
        f"Iteration {iter}: Best Cost = {best_cost[iter]}, Best Solution = {best_sol}")


Iteration 0: Best Cost = 42426.416067342594, Best Solution = [ 0.86898973 -0.86428436 -4.17200749 -3.3971065 ]
Iteration 1: Best Cost = 83666.00553549964, Best Solution = [ 0.86898973 -0.86428436 -4.17200749 -3.3971065 ]
Iteration 2: Best Cost = 83666.0055011478, Best Solution = [ 0.86898973 -0.86428436 -4.17200749 -3.3971065 ]
Iteration 3: Best Cost = 79372.54263623762, Best Solution = [ 0.86898973 -0.86428436 -4.17200749 -3.3971065 ]
Iteration 4: Best Cost = 52345.01654522907, Best Solution = [ 0.86898973 -0.86428436 -4.17200749 -3.3971065 ]
Iteration 5: Best Cost = 41472.892176889356, Best Solution = [ 0.86898973 -0.86428436 -4.17200749 -3.3971065 ]
Iteration 6: Best Cost = 63561.00028638145, Best Solution = [ 0.86898973 -0.86428436 -4.17200749 -3.3971065 ]


/usr/lib/python3/dist-packages/ipykernel_launcher.py:66: UserWarning: loadtxt: Empty input file: "pid_gain_err.txt"
/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/lib/python3/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

### Results

In [ ]:
fig = plt.figure(figsize=(18,8));
plt.semilogy(best_cost, linewidth=3, color='r');
plt.xlabel("Iteration");
plt.ylabel("Best Cost!");